## Problem
### Game of Life
Conway's Game of Life is a cellular automaton that is played on a grid of cells. Each cell can be either alive or dead. The game progresses in discrete steps. At each step, the following rules are applied to each cell:

- Any live cell with fewer than two live neighbours dies, as if by underpopulation.
- Any live cell with two or three live neighbours lives on to the next generation.
- Any live cell with more than three live neighbours dies, as if by overpopulation.
- Any dead cell with exactly three live neighbours becomes a live cell, as if by reproduction.

A cell's neighbours are those cells which are horizontally, vertically or diagonally adjacent. Most cells will have eight neighbours. We also follow wrapping conventions, so that the top row is adjacent to the bottom row, and the left column is adjacent to the right column.
Furthermore, we do not update the grid in place. This means we first compute the next state of the grid using the current state and then update the grid with the new state. Notably, this is often done using two arrays, one for the current state and one for the next state. This is because the next state of a cell depends on the current state of the cell and its neighbours. If we were to update the grid in place, then we would be using the new state of a cell to compute the new state of its neighbours, which is not what we want.

### Our parametrisation
For this challenge, we make a small change to the rules above. Instead of having a fixed number of neighbours for each of the rules, we allow the number of neighbours to be a parameter. In particular, we have three parameters A, B, and C and the rules are as follows:

- Any live cell with fewer than A live neighbours dies, as if by underpopulation.
- Any live cell with between A or B live neighbours lives on to the next generation (inclusive).
- Any live cell with more than B live neighbours dies, as if by overpopulation.
- Any dead cell with exactly C live neighbours becomes a live cell, as if by reproduction.

### Input
Your input will be passed in via standard in and will consist of several lines. The first line will be:

w h n m A B C, where:

- w is the width of the grid
- h is the height of the grid
- n is the number of steps to simulate
- m is the output frequency, i.e. the number of steps between each output
Then, A, B, and C, indicate the numbers for the rules mentioned above
We guarantee that the parameters will have the following bounds:

The next h lines will indicate the initial state. Each line will have w characters, each of which will be either . (dead) or # (alive).

### Output
The output must be the state of the grid, every m steps, for n steps. The output must be in the same format as the input. For instance, if n = 1001 and m = 250, we output the following:

The result after the first iteration
The result after the 251st iteration
The result after the 501st iteration
The result after the 751st iteration
The result after the 1001st iteration
You must always output the final state of the grid, but do not output it twice. Furthermore, the state after the first iteration must also always be printed, regardless of the value of m.

In [20]:
#Example 
# 10 10 1000000 100000000 2 3 3
# ..........
# .#........
# .#........
# ..........
# .....#..#.
# .......##.
# ........#.
# #.........
# ..........
# ..........

In [6]:
#Creating the print grid function
def printGrid(grid):
    bigGrid = []
    for i in grid:
        bigGrid.append(''.join(i))
    print('\n'.join(bigGrid))

#Creating the main game function
def conwaysGameOfLife(w, h, n, m, A, B, C, initial_grid):
    #Converting the initial grid to a list form
    #Separating into seperate rows, every 10 characters
    grid_splits = [initial_grid[i:i+w] for i in range(0, len(initial_grid), w)]
    #Seperating each row into different columns
    griddy = []
    for i in grid_splits:
        griddy.append(list(i))
    return griddy

In [7]:
printGrid(conwaysGameOfLife(10, 10, 1000000, 100000000, 2, 3, 3, '...........#.........#.......................#..#........##.........#.#.............................'))

..........
.#........
.#........
..........
.....#..#.
.......##.
........#.
#.........
..........
..........


In [18]:
test_grid = conwaysGameOfLife(10, 10, 1000000, 100000000, 2, 3, 3, '...........#.........#.......................#..#........##.........#.#.............................')

To make decisions on whether the cell will be dead or alive, we need the number of neighbors it has. To achieve this we need to iterate first get how many live neighbors a cell has. This can be done by iterating through each element in the list and keep track of row and column number

In [19]:
test_grid

[['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '#', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '#', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '#', '.', '.', '#', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '#', '#', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '#', '.'],
 ['#', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.']]

In [20]:
#Case 1
test_grid[0][10]

IndexError: list index out of range

In [21]:
#Case 2
test_grid[0][-11]

IndexError: list index out of range

Because each side of the grid is wrapped we have a problem with edge cells (Case 1). Hence we need to account for the case where our iterating variable is greater than the range of the index. This is only problem when we increment as the negative range will never be reached (Case 2)

In [22]:
#Neighbour mappings are: 

#  left-u |   up  | right-u
#  left-c | index | right-c
#  left-d |  down | right-d 

In [23]:
def cellHealth(value_arr):
    neighbor_result = []
    for i in value_arr:
        output = []
        for j in i:
            if j=='#':
                output.append(1)
            else:
                output.append(0)
        neighbor_result.append(output)
    return neighbor_result

In [36]:
def neighbour_checker(test_grid, i, j):
    i_down = i+1
    j_right = j+1

    if i == 9:
        i_down = 0
    if j == 9:
        j_right = 0
    
    #the current index is  [i][j]
    #Checking left-u neighbor
    lu = test_grid[i-1][j-1]
    #Checking left-c neighbour
    lc =test_grid[i][j-1]
    #Checking left_d neighbour
    ld = test_grid[i_down][j-1]

    #Checking up neighbour
    u = test_grid[i-1][j]
    #Checking down neighbour
    d = test_grid[i_down][j]

    #Checking right-u neighbour
    ru = test_grid[i-1][j_right]
    #Checking right-c neighbour
    rc = test_grid[i][j_right]
    #Checking right-d neighbour
    rd = test_grid[i_down][j_right]
    return cellHealth([[lu, lc , ld], [u, d], [ru, rc, rd]])

In [37]:
neighbour_checker(test2_grid, 1, 1)

[[1, 0, 0], [0, 1], [0, 0, 0]]

In [38]:
neighbour_checker(test2_grid, 9, 9)

[[0, 0, 0], [0, 0], [0, 0, 1]]

In [39]:
test2_grid = conwaysGameOfLife(10, 10, 1000000, 100000000, 2, 3, 3, '#..........#.........#.......................#..#........##.........#.#.............................')

In [40]:
test2_grid

[['#', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '#', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '#', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '#', '.', '.', '#', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '#', '#', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '#', '.'],
 ['#', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.']]